<a href="https://colab.research.google.com/github/tanvisingh18/Scroll---Emotion-Weighted-News-Summarizer/blob/main/Copy_of_Chnageswithdeberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q scikit-fuzzy newspaper3k lxml_html_clean feedparser sentence-transformers transformers sentencepiece vaderSentiment gTTS gradio tldextract nltk scikit-learn

import nltk
nltk.download('punkt')

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 34.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 9.8 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import os, uuid, time, math, tldextract, datetime as dt
import feedparser, pandas as pd, numpy as np
from newspaper import Article
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from gtts import gTTS
import gradio as gr
import skfuzzy as fuzz
from skfuzzy import control as ctrl

import torch
DEVICE = 0 if torch.cuda.is_available() else -1
print("Using device:", "GPU" if DEVICE==0 else "CPU")

Using device: GPU


In [ ]:
SUMMARIZER_MODEL = "sshleifer/distilbart-cnn-12-6"   # faster summarizer for Colab
EMOTION_MODEL = "j-hartmann/emotion-english-distilroberta-base"  # emotion classifier
EMBED_MODEL = "all-MiniLM-L6-v2"                    # embeddings for dedupe & semantic topic prompts

# Initialize pipelines & embedder
print("Loading models (can take a minute)...")
summarizer = pipeline("summarization", model=SUMMARIZER_MODEL, device=DEVICE)
emotion_pipe = pipeline("text-classification", model=EMOTION_MODEL, return_all_scores=True, device=DEVICE)
embedder = SentenceTransformer(EMBED_MODEL)

vader = SentimentIntensityAnalyzer()
print("Models loaded.")

Loading models (can take a minute)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Models loaded.


In [ ]:
# -------------------------
# RSS feed fetcher & full-text extraction
# -------------------------
import feedparser
import pandas as pd
import datetime as dt
import time
import uuid
import tldextract
from newspaper import Article

# List of RSS feeds
FEEDS = [
    "https://rss.cnn.com/rss/cnn_latest.rss",
    "http://feeds.bbci.co.uk/news/rss.xml",
    "https://feeds.reuters.com/reuters/topNews",
    "https://www.aljazeera.com/xml/rss/all.xml",
    "https://www.theguardian.com/world/rss"
]

def extract_full_text(url, fallback_text=""):
    """Return the full article text, but always append fallback_text (feed summary) so text is richer."""
    text = ""
    try:
        article = Article(url)
        article.download()
        article.parse()
        text = article.text.strip()
    except Exception:
        text = ""
    # Always append the feed-provided summary/description so short articles aren’t empty/identical
    combined = (text + "\n\n" + (fallback_text or "")).strip()
    return combined


def fetch_articles(max_per_feed=12, max_total=120):
    """Fetch articles from multiple RSS feeds and return a cleaned DataFrame."""
    rows = []

    for feed_url in FEEDS:
        feed_data = feedparser.parse(feed_url)

        for entry in feed_data.entries[:max_per_feed]:
            url = entry.get("link")
            title = (entry.get("title") or "").strip()
            summary = (entry.get("summary") or entry.get("description") or "").strip()

            # Parse publication date
            try:
                if entry.get("published_parsed"):
                    published_at = dt.datetime.fromtimestamp(time.mktime(entry.published_parsed))
                else:
                    published_at = dt.datetime.utcnow()
            except Exception:
                published_at = dt.datetime.utcnow()

            # Extract domain
            domain_parts = tldextract.extract(url)
            domain = f"{domain_parts.domain}.{domain_parts.suffix}" if domain_parts.suffix else domain_parts.domain

            # Extract full text with fallback
            full_text = extract_full_text(url, summary)

            # Skip entries with no title or content
            if not (title and (full_text or summary)):
                continue

            rows.append({
                "id": str(uuid.uuid4())[:8],
                "url": url,
                "domain": domain,
                "title": title,
                "raw": full_text,
                "published_at": published_at
            })

    # Build DataFrame and clean
    df = pd.DataFrame(rows).drop_duplicates(subset=["url"]).sort_values("published_at", ascending=False)
    df = df.head(max_total).reset_index(drop=True)
    df["hours_old"] = df["published_at"].apply(lambda d: (dt.datetime.utcnow() - d).total_seconds() / 3600.0)

    return df

# Quick fetch (example)
articles = fetch_articles(max_per_feed=20, max_total=400)
len(articles), articles.head(2)

/tmp/ipython-input-2449894886.py:80: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  df["hours_old"] = df["published_at"].apply(lambda d: (dt.datetime.utcnow() - d).total_seconds() / 3600.0)


(60,
          id                                                url   domain  \
 0  4a4f6e1e  https://www.bbc.com/news/articles/c8675l25591o...  bbc.com   
 1  46235f1b  https://www.bbc.com/news/articles/c8r007d4kk2o...  bbc.com   
 
                                                title  \
 0  King wants Andrew to lose final military title...   
 1  What we know about stabbings on Doncaster to L...   
 
                                                  raw        published_at  \
 0  King wants Andrew to lose final military title... 2025-11-02 10:25:06   
 1  What we know about stabbings on Doncaster to L... 2025-11-02 10:03:34   
 
    hours_old  
 0   0.104615  
 1   0.463504  )

In [ ]:
# -------------------------
# Deduplication and summarization helpers
# -------------------------
from sentence_transformers import util

def dedupe_titles(df, thresh=0.84):
    titles = df["title"].tolist()
    if not titles:
        return df
    embs = embedder.encode(titles, convert_to_tensor=True, normalize_embeddings=True)
    keep = []
    for i in range(len(titles)):
        skip = False
        for j in keep:
            if float(util.cos_sim(embs[i], embs[j])) >= thresh:
                skip = True; break
        if not skip:
            keep.append(i)
    return df.iloc[keep].reset_index(drop=True)

# Apply dedupe
articles = dedupe_titles(articles, thresh=0.92)

def make_summary(text, min_len=25, max_len=70):
    """
    Produce concise 3-5 line style summaries:
    - If text short, return raw text.
    - Otherwise request short summary with transformer.
    """
    text = text.strip()
    if len(text.split()) < 50:
        return text  # already short, no summarization needed
    # transformer summarizer; controlled lengths for compact summary
    out = summarizer(text, max_length=max_len, min_length=min_len, do_sample=False, truncation=True)
    return out[0]["summary_text"]

In [ ]:
# -------------------------
# Clustering
# -------------------------
from sklearn.cluster import AgglomerativeClustering

def cluster_articles(df, sim_threshold=0.65):
    if df.shape[0] == 0:
        df["cluster"] = []
        return df
    embs = embedder.encode(df["raw"].tolist(), normalize_embeddings=True)
    clustering = AgglomerativeClustering(
        n_clusters=None,
        distance_threshold=1 - sim_threshold,
        metric="cosine",
        linkage="average"
    ).fit(embs)
    df["cluster"] = clustering.labels_
    return df

articles = fetch_articles(8)   # small fetch for demo
articles = cluster_articles(articles)

def summarize_clusters(df):
    cluster_summaries = []
    for cluster_id, group in df.groupby("cluster"):
        combined_text = " ".join(group["raw"].tolist())[:4000]  # cap to avoid very long input
        summary_text = make_summary(combined_text)  # use safe summarizer
        cluster_summaries.append({
            "cluster_id": cluster_id,
            "topic": group.iloc[0]["title"],  # take first article title as representative
            "summary": summary_text,
            "sources": group["domain"].unique().tolist()
        })
    return pd.DataFrame(cluster_summaries)

clustered_summaries = summarize_clusters(articles)
clustered_summaries.head()


/tmp/ipython-input-2449894886.py:80: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  df["hours_old"] = df["published_at"].apply(lambda d: (dt.datetime.utcnow() - d).total_seconds() / 3600.0)
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


,cluster_id,topic,summary,sources
0,0,What we know about stabbings on Doncaster to L...,Attack took place on London North Eastern Rai...,"[bbc.com, aljazeera.com]"
1,1,"Era of free trade and investment is over, Cana...",Mark Carney warns that the era of free trade ...,[theguardian.com]
2,2,UN approves resolution supporting Morocco’s cl...,The resolution offers the strongest endorseme...,[theguardian.com]
3,3,‘How do you rebuild all this?’ Black River res...,"It is a treacherous journey to Black River, a...",[theguardian.com]
4,4,Sudan’s RSF accused of ‘PR stunt’ after arrest...,Sudan's paramilitary Rapid Support Forces cla...,[theguardian.com]


In [ ]:
# -------------------------
# Emotion scoring & fuzzy importance
# -------------------------
def get_emotion_scores(text):
    out = emotion_pipe(text[:400])
    if not out or not out[0]:
        return {}
    return {d["label"].lower(): float(d["score"]) for d in out[0]}


def fuzzify_membership(score):
    s = float(score)
    low = med = high = 0
    if s <= 0.25:
        low = 1.0
        med = s / 0.25
    elif s <= 0.5:
        low = (0.5 - s) / 0.25
        med = (s - 0.25) / 0.25
    elif s <= 0.75:
        med = (0.75 - s) / 0.25
        high = (s - 0.5) / 0.25
    else:
        high = 1.0
    return {"low": max(0, low), "med": max(0, med), "high": max(0, high)}


def compute_fuzzy_emotion_importance(em_scores, user_emotion_pref=None):
    base_weights = {
        "anger": 1.0, "sadness": 0.9, "fear": 0.8, "joy": 0.4,
        "surprise": 0.4, "neutral": 0.1, "love": 0.1,
        "optimism": 0.2, "disgust": 0.9
    }

    score = norm = 0.0
    for emo, val in em_scores.items():
        w = base_weights.get(emo, 0.3)
        mem = fuzzify_membership(val)

        # Combine memberships with reasonable weights
        mem_score = 0.25 * mem["low"] + 0.5 * mem["med"] + 1.0 * mem["high"]

        boost = 1.25 if user_emotion_pref and emo in user_emotion_pref else 1.0

        score += w * mem_score * boost
        norm += w * boost

    return float(score / norm) if norm > 0 else 0.0


# Apply emotion scoring on the cluster summaries
if not clustered_summaries.empty:
    clustered_summaries["emotion_scores"] = clustered_summaries["summary"].apply(get_emotion_scores)
    clustered_summaries["emo_importance"] = clustered_summaries["emotion_scores"].apply(compute_fuzzy_emotion_importance)
    clustered_summaries[["summary", "emotion_scores", "emo_importance"]].head(3)


In [ ]:
# -------------------------
# Profiles, TTS caching, Particle-style summarizer
# -------------------------
import os, json, math
from gtts import gTTS
from collections import Counter

USER_FILE = "/content/user_profiles.json"
AUDIO_DIR = "/content/audio"
os.makedirs(AUDIO_DIR, exist_ok=True)

# trusted domains for "Verified" tag
TRUSTED_SITES = {"cnn.com", "reuters.com", "bbc.co.uk", "nytimes.com", "theguardian.com",
                 "indiatimes.com", "aljazeera.com","hindustantimes.com","gulfnew.com",
                 "khaleejtimes.com","ndtv.com","thehindu.com"}

if not os.path.exists(USER_FILE):
    with open(USER_FILE, "w") as f:
        json.dump({}, f)

def load_user_profile(user_id="default"):
    with open(USER_FILE, "r") as f:
        allp = json.load(f)
    return allp.get(user_id, {"topics":{}, "emotions":{}, "clicks":{}, "seen":{}})

def save_user_profile(profile, user_id="default"):
    with open(USER_FILE, "r") as f:
        allp = json.load(f)
    allp[user_id] = profile
    with open(USER_FILE, "w") as f:
        json.dump(allp, f)

# --- TTS caching helper ---
def ensure_tts_for_summary(cluster_id, text):
    path = os.path.join(AUDIO_DIR, f"{cluster_id}.mp3")
    if os.path.exists(path):
        return path
    try:
        gTTS(text=text[:3000], lang="en").save(path)
        return path
    except Exception:
        open(path, "wb").close()
        return path

# --- enhanced cluster summarizer (Particle-style) ---
def summarize_clusters_particle(df, min_sources=10, max_bullets=20):
    cluster_cards = []
    for cluster_id, group in df.groupby("cluster"):
        group_for_overview = group.head(min_sources) if len(group) >= min_sources else group
        combined_text = " ".join(group_for_overview["raw"].tolist())[:6000]
        overview = make_summary(combined_text, min_len=40, max_len=120)

        bullets = []
        for _, row in group.head(max_bullets).iterrows():
            mini = make_summary(row["raw"], min_len=15, max_len=25)
            bullets.append(f"{row['domain']}: {mini}")

        domain_counts = Counter(group["domain"].tolist())
        sources_list = [f"{d} ({c})" for d,c in domain_counts.most_common()]

        cluster_cards.append({
            "cluster_id": cluster_id,
            "overview": overview,
            "bullets": bullets,
            "sources": sources_list,
            "urls": group["url"].tolist(),
            "title": group.iloc[0]["title"]
        })
    return pd.DataFrame(cluster_cards)

# --- rebuild cluster summaries in Particle style ---
clustered_summaries = summarize_clusters_particle(articles)

# add emotion, recency, verified, audio
cluster_to_hours = articles.groupby("cluster")["hours_old"].min().to_dict()
clustered_summaries["hours_old"] = clustered_summaries["cluster_id"].map(cluster_to_hours).fillna(999)
clustered_summaries["recency"] = clustered_summaries["hours_old"].apply(lambda h: max(0, 1 - min(h,72)/72))
clustered_summaries["verified"] = clustered_summaries["sources"].apply(lambda s: any(site in TRUSTED_SITES for site in s))
clustered_summaries["emotion_scores"] = clustered_summaries["overview"].apply(get_emotion_scores)
clustered_summaries["emo_importance"] = clustered_summaries["emotion_scores"].apply(compute_fuzzy_emotion_importance)
clustered_summaries["audio"] = clustered_summaries.apply(lambda r: ensure_tts_for_summary(r["cluster_id"], r["overview"]), axis=1)
clustered_summaries["clicks_global"] = 0


Your max_length is set to 120, but your input_length is only 89. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=44)
Your max_length is set to 120, but your input_length is only 84. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=42)


In [ ]:
# -------------------------
# 🧠 Semantic embedding-based Topic Classification (REPLACED heavy zero-shot)
# -------------------------
print("Preparing semantic topic prompt embeddings...")

CANDIDATE_TOPICS = ["technology", "politics", "business", "world", "sports", "science", "health", "entertainment"]

topic_prompts = {
    "world": "international and global news about countries, wars, diplomacy, elections",
    "sports": "sports, games, tournaments, matches, players, teams, competitions",
    "technology": "news about technology, gadgets, AI, software, hardware, startups, innovation",
    "politics": "government, elections, policies, leaders, parliament, laws, protests",
    "business": "companies, markets, economy, finance, investments, stock market, trade",
    "science": "discoveries, space, research, experiments, scientists, innovations",
    "health": "medicine, diseases, healthcare, hospitals, fitness, vaccines",
    "entertainment": "movies, music, celebrities, television, pop culture, arts"
}

topic_texts = list(topic_prompts.values())
topic_embs = embedder.encode(topic_texts, normalize_embeddings=True)

def infer_topic(text):
    """Infer topic based on semantic similarity between article text and topic descriptions."""
    try:
        text_emb = embedder.encode(text, normalize_embeddings=True)
        sims = util.cos_sim(text_emb, topic_embs)[0].cpu().tolist()
        best_idx = sims.index(max(sims))
        return list(topic_prompts.keys())[best_idx]
    except Exception:
        return "world"

# Apply topic inference
clustered_summaries["topic"] = clustered_summaries["overview"].apply(infer_topic)


Preparing semantic topic prompt embeddings...


In [ ]:
# -------------------------
# --- Fix: Short summaries, content-type classification and daily digest
# -------------------------
# Lightweight content-type classifier
def classify_content_type(text):
    text_lower = text.lower()
    if any(k in text_lower for k in ["breaking", "urgent", "just in"]):
        return "breaking news"
    if any(k in text_lower for k in ["opinion", "analysis", "editorial", "column"]):
        return "opinion/editorial"
    if any(k in text_lower for k in ["update", "developing story", "live coverage"]):
        return "developing"
    return "report"

# Daily Digest Mode (Top 10 stories)
def daily_digest(df, profile, top_n=10):
    ranked = build_ranked_feed(profile, df)
    top_stories = ranked.head(top_n)
    digest = "\n\n".join([f"📰 {r['title']} — {r['overview']}" for _, r in top_stories.iterrows()])
    try:
        gTTS(digest[:3000], lang="en").save(f"{AUDIO_DIR}/daily_digest.mp3")
        audio_path = f"{AUDIO_DIR}/daily_digest.mp3"
    except Exception:
        audio_path = None
    # Return a simple HTML-safe string (Gradio HTML can render raw HTML)
    digest_html = "<div style='font-family:Inter, sans-serif; color:#e5e7eb; line-height:1.6;'>" + \
                  "<h3>Daily Digest</h3>" + \
                  "<p>" + "</p><p>".join([r['title'] + " — " + r['overview'] for _, r in top_stories.iterrows()]) + \
                  "</p></div>"
    return digest_html, audio_path

# Apply new labels and features
clustered_summaries["content_type"] = clustered_summaries["overview"].apply(classify_content_type)


In [ ]:
# -------------------------
# --- Fix: Short summaries, content-type classification and daily digest
# -------------------------
# Lightweight content-type classifier
def classify_content_type(text):
    text_lower = text.lower()
    if any(k in text_lower for k in ["breaking", "urgent", "just in"]):
        return "breaking news"
    if any(k in text_lower for k in ["opinion", "analysis", "editorial", "column"]):
        return "opinion/editorial"
    if any(k in text_lower for k in ["update", "developing story", "live coverage"]):
        return "developing"
    return "report"

# Daily Digest Mode (Top 10 stories)
def daily_digest(df, profile, top_n=10):
    ranked = build_ranked_feed(profile, df)
    top_stories = ranked.head(top_n)
    digest = "\n\n".join([f"📰 {r['title']} — {r['overview']}" for _, r in top_stories.iterrows()])
    try:
        gTTS(digest[:3000], lang="en").save(f"{AUDIO_DIR}/daily_digest.mp3")
        audio_path = f"{AUDIO_DIR}/daily_digest.mp3"
    except Exception:
        audio_path = None
    # Return a simple HTML-safe string (Gradio HTML can render raw HTML)
    digest_html = "<div style='font-family:Inter, sans-serif; color:#e5e7eb; line-height:1.6;'>" + \
                  "<h3>Daily Digest</h3>" + \
                  "<p>" + "</p><p>".join([r['title'] + " — " + r['overview'] for _, r in top_stories.iterrows()]) + \
                  "</p></div>"
    return digest_html, audio_path

# Apply new labels and features
clustered_summaries["content_type"] = clustered_summaries["overview"].apply(classify_content_type)


In [ ]:
# -------------------------
# --- Personalization & ranking
# -------------------------
profile = load_user_profile("default")
if not profile.get("topics"):
    profile["topics"] = {t: (1.0 if i < 2 else 0.1) for i,t in enumerate(CANDIDATE_TOPICS)}

def score_row_for_user(row, profile):
    topic_pref = profile["topics"].get(row.get("topic","world"), 0.0)
    recency = row["recency"]; emo_importance = row["emo_importance"]
    verified = 1.0 if row["verified"] else 0.0
    return 2*topic_pref + 1.4*emo_importance + 1.0*recency + 0.3*verified

def build_ranked_feed(profile, df):
    dfc = df.copy()
    dfc["rank_score"] = dfc.apply(lambda r: score_row_for_user(r, profile), axis=1)
    return dfc.sort_values("rank_score", ascending=False).reset_index(drop=True)

STATE = {"idx": 0, "df": build_ranked_feed(profile, clustered_summaries), "profile": profile}


In [ ]:
# -------------------------
# --- Render card & navigation
# -------------------------
def render_card(idx):
    df = STATE["df"]
    if df.shape[0]==0:
        return ("No stories", None, "0/0", "")
    idx = max(0, min(idx, len(df)-1))
    r = df.iloc[idx]
    html = f"""
<div style="
  background: rgba(255,255,255,0.03);
  border-radius: 18px;
  padding: 22px;
  box-shadow: 0 0 15px rgba(255,255,255,0.05);
  font-family: 'Inter', sans-serif;
  color: #e5e7eb;
  line-height: 1.6;
">
  <h2 style="color:#a78bfa; font-size:1.4em; font-weight:700; margin-bottom:10px;">
    {r['title']}
  </h2>

  <p style="margin-top:6px; font-size:15px; color:#d1d5db;">
    {r['overview']}
  </p>

  <div style="margin-top:14px; font-weight:600; color:#93c5fd;">Topic: {r['topic'].capitalize()}</div>
  <div style="margin-top:6px; font-size:12px; color:#9ca3af;">Type: {r.get('content_type','report')}</div>

  <div style="margin-top:14px; font-weight:600; color:#93c5fd;">What others are saying:</div>
  <ul style="list-style:none; padding-left:0; margin-top:6px;">
    {''.join([f"<li style='margin-bottom:4px; color:#9ca3af;'>• {b}</li>" for b in r['bullets']])}
  </ul>

  <div style="margin-top:10px; font-size:12px; color:#9ca3af;">
    📰 Sources: {', '.join(r['sources'])}
  </div>

  <a href="{r['urls'][0]}" target="_blank"
     style="display:inline-block; margin-top:12px; padding:8px 16px;
            background:linear-gradient(90deg,#6366f1,#8b5cf6);
            border-radius:10px; color:white; text-decoration:none;
            font-weight:600; transition:all 0.2s ease-in-out;">
     🔗 Open Original
  </a>
</div>
"""
    return html, r["audio"], f"{idx+1}/{len(df)}", str(r["cluster_id"])

def next_card(topics):
    STATE["idx"]=min(STATE["idx"]+1,len(STATE["df"])-1); return render_card(STATE["idx"])
def prev_card(topics):
    STATE["idx"]=max(STATE["idx"]-1,0); return render_card(STATE["idx"])

def apply_topic_filters(topics):
    p = STATE["profile"]
    p["topics"] = {t:(1.0 if t in topics else 0.1) for t in CANDIDATE_TOPICS}
    save_user_profile(p,"default"); STATE["profile"]=p
    if topics:
        filtered = clustered_summaries[clustered_summaries["topic"].isin(topics)]
        if filtered.empty:
            filtered = clustered_summaries
    else:
        filtered = clustered_summaries
    STATE["df"] = build_ranked_feed(p, filtered)
    STATE["idx"] = 0
    return render_card(0)


In [ ]:
# -------------------------
# --- Frontend (Gradio) ---
# -------------------------
custom_css = """
body {
  background: radial-gradient(circle at top left, #0a0a0a, #111111);
  font-family: 'Inter', sans-serif;
  color: white;
}
.gradio-container {
  background: transparent !important;
}
.gr-block {
  background: rgba(255,255,255,0.04);
  backdrop-filter: blur(20px);
  border-radius: 18px !important;
  box-shadow: 0 0 20px rgba(255,255,255,0.05);
  padding: 15px;
}
h1, h2, h3 {
  color: #e2e8f0;
  text-shadow: 0 0 10px rgba(0,0,255,0.2);
}
button {
  background: linear-gradient(90deg, #6366f1, #8b5cf6) !important;
  border: none !important;
  color: white !important;
  font-weight: 600 !important;
  border-radius: 12px !important;
  transition: all 0.2s ease-in-out !important;
}
button:hover {
  transform: scale(1.05);
  box-shadow: 0 0 10px #8b5cf6;
}
div, button, .gradio-container {
  transition: all 0.3s ease-in-out;
}
"""

with gr.Blocks(css=custom_css, theme=gr.themes.Soft(primary_hue="violet")) as demo:
    gr.Markdown("""
    <h1 style='text-align:center; color:#a78bfa; font-size:2.2em;'>
    🪶 Particle AI News Feed
    </h1>
    <p style='text-align:center; color:#9ca3af;'>
    Your personalized, emotion-aware, topic-driven news assistant
    </p>
    """)

    topics = gr.CheckboxGroup(
        CANDIDATE_TOPICS,
        value=[t for t,v in STATE["profile"]["topics"].items() if v>0.9],
        label="Select Topics You Care About"
    )

    html_out = gr.HTML()
    audio_out = gr.Audio(interactive=False, type="filepath")
    pos_txt = gr.Textbox(interactive=False)
    hidden_cluster = gr.Textbox(value="", visible=False)

    with gr.Row():
        prev_btn = gr.Button("⬅ Prev", scale=1)
        next_btn = gr.Button("Next ➡", scale=1)
        digest_btn = gr.Button("📅 Daily Digest", scale=1)
        open_btn = gr.Button("🔗 Open Original", scale=1)

    topics.change(fn=apply_topic_filters, inputs=[topics], outputs=[html_out, audio_out, pos_txt, hidden_cluster])
    prev_btn.click(fn=prev_card, inputs=[topics], outputs=[html_out, audio_out, pos_txt, hidden_cluster])
    next_btn.click(fn=next_card, inputs=[topics], outputs=[html_out, audio_out, pos_txt, hidden_cluster])
    # daily digest returns (html, audio_path)
    digest_btn.click(fn=lambda: daily_digest(STATE["df"], STATE["profile"]), inputs=None, outputs=[html_out, audio_out])
    open_btn.click(fn=lambda cid: render_card(STATE["idx"]), inputs=[hidden_cluster], outputs=[html_out, audio_out, pos_txt, hidden_cluster])

    h, a, p, cid = render_card(0)
    html_out.value = h
    audio_out.value = a
    pos_txt.value = p
    hidden_cluster.value = cid

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ff32a7d853f528bf5f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# -------------------------
# --- Optional: Additional semantic topic classifier debug helpers (kept for development)
# -------------------------
# Re-use embedder if needed (already loaded earlier)
# topic_embs already prepared above.

def debug_topic(text):
    text_emb = embedder.encode(text, normalize_embeddings=True)
    sims = util.cos_sim(text_emb, topic_embs)[0].cpu().tolist()
    scores = dict(zip(topic_prompts.keys(), sims))
    return sorted(scores.items(), key=lambda x: x[1], reverse=True)

# quick tests
print(debug_topic("Elon Musk unveils new Tesla AI robot"))
print(debug_topic("Bangladesh’s fugitive ex-leader warns of mass voter boycott in 2026 polls"))


[('technology', 0.2867389917373657), ('science', 0.1265275627374649), ('world', 0.05382826179265976), ('politics', 0.029767408967018127), ('business', 0.002309070900082588), ('health', 0.0022393856197595596), ('entertainment', -0.03013920597732067), ('sports', -0.10291780531406403)]
[('politics', 0.26035282015800476), ('world', 0.16422006487846375), ('technology', 0.016840854659676552), ('entertainment', 0.011532505974173546), ('sports', -0.041737861931324005), ('science', -0.048380956053733826), ('business', -0.06871043145656586), ('health', -0.06910795718431473)]
